#install the depencencies

In [1]:
import quandl
import numpy as np
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

SyntaxError: invalid syntax (<ipython-input-6-960d07ba9c52>, line 1)

In [48]:
df=quandl.get("WIKI/FB")#"WIKI/AWZN" OR ../GOOGLE/ANY STOCK

ConnectionError: HTTPSConnectionPool(host='www.quandl.com', port=443): Max retries exceeded with url: /api/v3/datasets/WIKI/FB/data?order=asc (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000026B04CF7A58>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [9]:
print(df.head())

             Open   High    Low    Close       Volume  Ex-Dividend  \
Date                                                                 
2012-05-18  42.05  45.00  38.00  38.2318  573576400.0          0.0   
2012-05-21  36.53  36.66  33.00  34.0300  168192700.0          0.0   
2012-05-22  32.61  33.59  30.94  31.0000  101786600.0          0.0   
2012-05-23  31.37  32.50  31.36  32.0000   73600000.0          0.0   
2012-05-24  32.95  33.21  31.77  33.0300   50237200.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2012-05-18          1.0      42.05      45.00     38.00     38.2318   
2012-05-21          1.0      36.53      36.66     33.00     34.0300   
2012-05-22          1.0      32.61      33.59     30.94     31.0000   
2012-05-23          1.0      31.37      32.50     31.36     32.0000   
2012-05-24          1.0      32.95      33.21     31.77     33.0300   

           

In [13]:
#get the adjusted close price
df=df[['Adj. Close']]
print(df.head())

            Adj. Close
Date                  
2012-05-18     38.2318
2012-05-21     34.0300
2012-05-22     31.0000
2012-05-23     32.0000
2012-05-24     33.0300


In [19]:
#a variable for predicting 'n' days out into future
forecast_out=30 #forrecost value may be 1day,2days,45 days
#create another column (the target or depedent variable),shifted 'n' unit up
df['Predection']=df[['Adj. Close']].shift(-forecast_out)
print(df.head())

            Adj. Close  Predection
Date                              
2012-05-18     38.2318      30.771
2012-05-21     34.0300      31.200
2012-05-22     31.0000      31.470
2012-05-23     32.0000      31.730
2012-05-24     33.0300      32.170


In [25]:
#create independent dataset(x)
x=np.array(df.drop(['Predection'],1))
#remove the last'n' rows
x=x[:-forecast_out]

In [28]:
#creating the dependent dataset(y)
y=np.array(df['Predection'])

In [29]:
y

array([30.771, 31.2  , 31.47 , ...,    nan,    nan,    nan])

In [30]:
#remove nan value from y
y=y[:-forecast_out]

In [31]:
y

array([ 30.771,  31.2  ,  31.47 , ..., 159.39 , 160.06 , 152.19 ])

In [32]:
#spilite a data into 80%train and 20% test data set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [33]:
#create and train the svm (regressor)
svr_rbf=SVR(kernel='rbf',C=1e3,gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [34]:
#testing MOdel: score return the cofficient of determination R^2 of the prediction
svm_confidence=svr_rbf.score(x_test,y_test)
print("svm confidence:",svm_confidence)

svm confidence: 0.9846535346713066


In [35]:
#crete and train the linear Regression model
lr=LinearRegression()

#train the model
lr.fit(x_train,y_train)
lr_confidence=lr.score(x_test,y_test)
print("lr confidence:",lr_confidence)

lr confidence: 0.9801049198282453


In [38]:
#set the X_forecost equal to the last30 rows of the original dataset from Adj. cLOse
x_forecost=np.array(df.drop(['Predection'],1))[-forecast_out:]

In [39]:
print(x_forecost)

[[173.15]
 [179.52]
 [179.96]
 [177.36]
 [176.01]
 [177.91]
 [178.99]
 [183.29]
 [184.93]
 [181.46]
 [178.32]
 [175.94]
 [176.62]
 [180.4 ]
 [179.78]
 [183.71]
 [182.34]
 [185.23]
 [184.76]
 [181.88]
 [184.19]
 [183.86]
 [185.09]
 [172.56]
 [168.15]
 [169.39]
 [164.89]
 [159.39]
 [160.06]
 [152.19]]


In [46]:
#print linear regression model for next n days
lr_Predection=lr.predict(x_forecost)
print(lr_Predection)

[177.066543   183.49522824 183.93928186 181.31532869 179.95289148
 181.87039571 182.96034548 187.29996033 188.95506925 185.45310099
 182.28417294 179.88224658 180.56851126 184.38333547 183.75762356
 187.72382969 186.34120821 189.25783307 188.78350308 185.87697035
 188.20825181 187.8752116  189.11654329 176.47110747 172.02047922
 173.27190304 168.73044564 163.17977549 163.85594804 155.91344366]


In [47]:
#print suport vector regression model for next n days
svr_rbf_Predection=svr_rbf.predict(x_forecost)
print(svr_rbf_Predection)

[174.69789411 179.46482629 178.60047695 178.30347783 175.18666298
 180.05558306 180.61287821 186.85180363 179.53874263 182.69002953
 180.84016732 175.14551462 176.06208868 178.49382598 178.89579447
 184.78164545 187.29642164 179.50245011 179.82859694 185.20041788
 182.07359707 183.92484094 179.44537645 175.52766277 170.9429546
 175.0404653  172.86983837 167.45582854 166.31166519 159.5646348 ]
